In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession
spark= SparkSession.builder.getOrCreate()

In [12]:
training_data = spark.read.csv("D:\\spring semester - 2018\\Big Data Analytics\\HW_final\\1a_training.csv").rdd
test_data = spark.read.csv("D:\\spring semester - 2018\\Big Data Analytics\\HW_final\\1a_test.csv").rdd

In [13]:
trainingRDD = training_data.map(lambda p: Row(userId= int(p[0]), movieId= int(p[1]),rating= int(p[2])))
train = spark.createDataFrame(trainingRDD)

testingRDD = test_data.map(lambda p: Row(userId= int(p[0]), movieId= int(p[1]),rating= int(p[2])))
test = spark.createDataFrame(testingRDD)

In [62]:
als = ALS(rank = 2, maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",coldStartStrategy="drop")

In [77]:
model = als.fit(train)

In [64]:
predictions = model.transform(test)

In [67]:
predict = predictions.toPandas()

In [68]:
predict

,movieId,rating,userId,prediction
0,12,5,18,3.298120
1,1,2,5,2.153019
2,1,6,9,4.236707
3,1,7,4,25.277845
4,1,7,10,3.925784
5,13,7,10,4.036488
6,16,5,6,6.461028
7,16,6,19,-4.458359
8,16,1,4,32.371510
9,16,1,25,18.968132


In [69]:
predict['difference'] = predict[['rating', 'prediction']].apply(lambda x: ''.join(str(x)), axis=1)

In [70]:
predict['difference'] = predict['rating'].astype(str) +","+ predict['prediction'].astype(str)

In [72]:
predict.pivot(index = 'userId', columns = 'movieId', values = 'difference').to_csv("D:\\spring semester - 2018\\Big Data Analytics\\HW_final\\1dprediction.csv")

In [76]:
from sklearn.metrics import mean_squared_error

b = mean_squared_error(predict['prediction'],predict['rating'])

In [75]:
b

42.108735287094206